In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from pathlib import Path
import string
import re
import joblib
import json
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

/Users/Admin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def loadJson(jsonFile):
    with open(jsonFile) as file:
        JsonData = json.loads(file.read())
    return JsonData

In [3]:
data=loadJson("intent.json")

In [4]:
def pullData(feat_1,feat_2,is_pattern):
    is_pattern = is_pattern
    df = pd.DataFrame(columns=[feat_1,feat_2])
    for intent in data['intents']:
        if is_pattern:
            for pattern in intent['text']:
                w = pattern
                df_to_append = pd.Series([w,intent['intent']], index = df.columns)
                df = df.append(df_to_append,ignore_index=True)
        else:
            for response in intent['responses']:
                w = response
                df_to_append = pd.Series([w,intent['intent']], index = df.columns)
                df = df.append(df_to_append,ignore_index=True)
    return df

In [5]:
classes = []
for intent in data['intents']:
    for pattern in intent['text']:
        #tokenize each word
        if intent['intent'] not in classes:
            classes.append(intent['intent'])

In [6]:
classes = sorted(list(set(classes)))
joblib.dump(classes,'classes.pkl')

['classes.pkl']

In [7]:
data1=pullData('questions','labels',True)
data1.head(5)

,questions,labels
0,Hi,Greeting
1,Hi there,Greeting
2,Hola,Greeting
3,Hello,Greeting
4,Hello there,Greeting


In [8]:
data2=pullData('response','labels',False)
data2.head(5)

,response,labels
0,"Hi human, please tell me your GeniSys user",Greeting
1,"Hello human, please tell me your GeniSys user",Greeting
2,"Hola human, please tell me your GeniSys user",Greeting
3,Great! Hi <HUMAN>! How can I help?,GreetingResponse
4,"Good! Hi <HUMAN>, how can I help you?",GreetingResponse


In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Admin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
lemmatizer = WordNetLemmatizer()
vocab = Counter()
labels = []
def tokenizer(entry):
    tokens = entry.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('', w) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens]
    tokens = [word.lower() for word in tokens if len(word) > 1]
    return tokens

In [11]:
def stopWords(tokenizer,df,feature):
    withoutStopwords = []
    for entry in df[feature]:
        tokens = tokenizer(entry)
        joblib.dump(tokens,'tokens.pkl')
        withoutStopwords.append(' '.join(tokens))
    df[feature] = withoutStopwords
    return

In [12]:
def create(tokenizer,df,feature):
    for entry in df[feature]:
        tokens = tokenizer(entry)   
        vocab.update(tokens)
    joblib.dump(vocab,'vocab.pkl')
    return

In [13]:
create(tokenizer,data1,'questions')
stopWords(tokenizer,data1,'questions')

In [14]:
data1.head(10)

,questions,labels
0,hi,Greeting
1,hi there,Greeting
2,hola,Greeting
3,hello,Greeting
4,hello there,Greeting
5,hya,Greeting
6,hya there,Greeting
7,my user is adam,GreetingResponse
8,this is adam,GreetingResponse
9,am adam,GreetingResponse


In [15]:
test_list = list(data1.groupby(by='labels',as_index=False).first()['questions'])
test_list

['you are very clever',
 'thanks bye',
 'how are you',
 'good thanks my user is adam',
 'what is my name',
 'bye',
 'am bored gossip with me',
 'hi',
 'my user is adam',
 'tell me joke',
 'what is your name',
 'am not talking to you',
 'open the pod bay door',
 'why',
 'what is your real name',
 'can you prove you are selfaware',
 'be quiet',
 'fuck off',
 'ok thank you',
 'what is the time',
 'do you understand what am saying',
 'can you see me']

In [16]:
test_index = []
for i,_ in enumerate(test_list):
    idx = data1[data1.questions == test_list[i]].index[0]
    test_index.append(idx)
test_index

[103,
 91,
 15,
 22,
 30,
 87,
 110,
 0,
 7,
 116,
 37,
 63,
 122,
 129,
 43,
 136,
 76,
 83,
 57,
 50,
 70,
 97]

In [17]:
train_index = [i for i in data1.index if i not in test_index]

In [18]:
def encoder(df,feature):
    t = Tokenizer()
    entries = [entry for entry in df[feature]]
    t.fit_on_texts(entries)
    joblib.dump(t,'tokenizer_t.pkl')
    vocab_size = len(t.word_index) + 1
    entries = [entry for entry in df[feature]]
    max_length = max([len(s.split()) for s in entries])
    encoded = t.texts_to_sequences(entries)
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded, vocab_size

In [19]:
padded,vocab_size = encoder(data1,'questions')

In [20]:
encoded=pd.DataFrame(padded)

In [21]:
encoded['labels']=data1.labels
encoded.head(5)


,0,1,2,3,4,5,6,7,8,labels
0,47,0,0,0,0,0,0,0,0,Greeting
1,47,48,0,0,0,0,0,0,0,Greeting
2,61,0,0,0,0,0,0,0,0,Greeting
3,36,0,0,0,0,0,0,0,0,Greeting
4,36,48,0,0,0,0,0,0,0,Greeting


In [22]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [23]:
labell=labelencoder.fit_transform(encoded.labels)
labell

array([ 7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  2,  2,
        2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4, 10, 10, 10, 10, 10, 10, 14, 14, 14, 14, 14, 14, 14, 19,
       19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 11, 11, 11, 11, 11,
       11, 11, 20, 20, 20, 20, 20, 20, 16, 16, 16, 16, 16, 16, 16, 17, 17,
       17, 17,  5,  5,  5,  5,  1,  1,  1,  1,  1,  1, 21, 21, 21, 21, 21,
       21,  0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  6,  6,  6,  9,  9,  9,
        9,  9,  9, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13,
       15, 15, 15, 15, 15, 15, 15])

In [24]:
mapper = {}
for index,key in enumerate(encoded.labels):
    if key not in mapper.keys():
        mapper[key] = labell[index]
mapper

{'Greeting': 7,
 'GreetingResponse': 8,
 'CourtesyGreeting': 2,
 'CourtesyGreetingResponse': 3,
 'CurrentHumanQuery': 4,
 'NameQuery': 10,
 'RealNameQuery': 14,
 'TimeQuery': 19,
 'Thanks': 18,
 'NotTalking2U': 11,
 'UnderstandQuery': 20,
 'Shutup': 16,
 'Swearing': 17,
 'GoodBye': 5,
 'CourtesyGoodBye': 1,
 'WhoAmI': 21,
 'Clever': 0,
 'Gossip': 6,
 'Jokes': 9,
 'PodBayDoor': 12,
 'PodBayDoorResponse': 13,
 'SelfAware': 15}

In [25]:
data2.labels = data2.labels.map(mapper).astype({'labels': 'int32'})
data2.head()

,response,labels
0,"Hi human, please tell me your GeniSys user",7
1,"Hello human, please tell me your GeniSys user",7
2,"Hola human, please tell me your GeniSys user",7
3,Great! Hi <HUMAN>! How can I help?,8
4,"Good! Hi <HUMAN>, how can I help you?",8


In [26]:
data2.to_csv('response.csv',index=False)

In [27]:
train = encoded.loc[train_index]
test = encoded.loc[test_index]

In [28]:
X_train=train.drop(columns=['labels'],axis=1)
y_train=train.labels
X_test=test.drop(columns=['labels'],axis=1)
y_test=test.labels

In [29]:
y_train =pd.get_dummies(y_train).values
y_test =pd.get_dummies(y_test).values

In [30]:
max_length=X_train.shape[1]

(121, 22)

In [46]:

def define_model(vocab_size,max_length):
    model = Sequential()
    model.add(Embedding(vocab_size,300, input_length=max_length))
    model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
    model.add(Flatten())
#     model.add(Dense(32, activation='relu'))
    model.add(Dense(22, activation='softmax'))
    
    
    # compile network
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss = 'categorical_crossentropy',
              # optimizer = Adam(lr=0.001),
              optimizer = 'adam',
              metrics = ['accuracy'])
    
    # summarize defined model
    model.summary()
#     plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [32]:
model = define_model(vocab_size,max_length)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9, 300)            34500     
_________________________________________________________________
conv1d (Conv1D)              (None, 6, 64)             76864     
_________________________________________________________________
flatten (Flatten)            (None, 384)               0         
_________________________________________________________________
dense (Dense)                (None, 22)                8470      
Total params: 119,834
Trainable params: 119,834
Non-trainable params: 0
_________________________________________________________________


In [33]:
history = model.fit(X_train, y_train, epochs=500, verbose=1,validation_data=(X_test,y_test))

/Users/Admin/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 121 samples, validate on 22 samples
Epoch 1/500
121/121 [==============================] - 2s 18ms/step - loss: 3.0871 - acc: 0.0744 - val_loss: 3.0579 - val_acc: 0.1818
Epoch 2/500
121/121 [==============================] - 0s 1ms/step - loss: 3.0039 - acc: 0.3306 - val_loss: 3.0167 - val_acc: 0.2727
Epoch 3/500
121/121 [==============================] - 0s 1ms/step - loss: 2.9250 - acc: 0.4380 - val_loss: 2.9637 - val_acc: 0.2727
Epoch 4/500
121/121 [==============================] - 0s 1ms/step - loss: 2.8310 - acc: 0.5289 - val_loss: 2.9013 - val_acc: 0.2727
Epoch 5/500
121/121 [==============================] - 0s 1ms/step - loss: 2.7154 - acc: 0.6116 - val_loss: 2.8247 - val_acc: 0.3636
Epoch 6/500
121/121 [==============================] - 0s 1ms/step - loss: 2.5900 - acc: 0.6777 - val_loss: 2.7378 - val_acc: 0.3636
Epoch 7/500
121/121 [==============================] - 0s 1ms/step - loss: 2.4318 - acc: 0.7438 - val_loss: 2.6385 - val_acc: 0.4545
Epoch 8/500
121/121 [==

Epoch 62/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0128 - acc: 1.0000 - val_loss: 0.6506 - val_acc: 0.7273
Epoch 63/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0122 - acc: 1.0000 - val_loss: 0.6512 - val_acc: 0.7273
Epoch 64/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0117 - acc: 1.0000 - val_loss: 0.6460 - val_acc: 0.7273
Epoch 65/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0112 - acc: 1.0000 - val_loss: 0.6436 - val_acc: 0.7273
Epoch 66/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0107 - acc: 1.0000 - val_loss: 0.6426 - val_acc: 0.7273
Epoch 67/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0102 - acc: 1.0000 - val_loss: 0.6469 - val_acc: 0.7273
Epoch 68/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0099 - acc: 1.0000 - val_loss: 0.6455 - val_acc: 0.7273
Epoch 69/500
121/121 [==============================] - 0s 1ms

Epoch 123/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.6069 - val_acc: 0.7727
Epoch 124/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.6084 - val_acc: 0.7273
Epoch 125/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0023 - acc: 1.0000 - val_loss: 0.6084 - val_acc: 0.7273
Epoch 126/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.6068 - val_acc: 0.7273
Epoch 127/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.6076 - val_acc: 0.7273
Epoch 128/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.6068 - val_acc: 0.7273
Epoch 129/500
121/121 [==============================] - 0s 1ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.6065 - val_acc: 0.7727
Epoch 130/500
121/121 [==============================] 

121/121 [==============================] - 0s 1ms/step - loss: 9.9241e-04 - acc: 1.0000 - val_loss: 0.5972 - val_acc: 0.7727
Epoch 184/500
121/121 [==============================] - 0s 1ms/step - loss: 9.8081e-04 - acc: 1.0000 - val_loss: 0.5962 - val_acc: 0.7727
Epoch 185/500
121/121 [==============================] - 0s 1ms/step - loss: 9.6925e-04 - acc: 1.0000 - val_loss: 0.5962 - val_acc: 0.7727
Epoch 186/500
121/121 [==============================] - 0s 1ms/step - loss: 9.6039e-04 - acc: 1.0000 - val_loss: 0.5947 - val_acc: 0.7727
Epoch 187/500
121/121 [==============================] - 0s 1ms/step - loss: 9.4807e-04 - acc: 1.0000 - val_loss: 0.5952 - val_acc: 0.7727
Epoch 188/500
121/121 [==============================] - 0s 1ms/step - loss: 9.3665e-04 - acc: 1.0000 - val_loss: 0.5948 - val_acc: 0.7727
Epoch 189/500
121/121 [==============================] - 0s 1ms/step - loss: 9.2662e-04 - acc: 1.0000 - val_loss: 0.5947 - val_acc: 0.7727
Epoch 190/500
121/121 [==================

121/121 [==============================] - 0s 1ms/step - loss: 5.5507e-04 - acc: 1.0000 - val_loss: 0.5954 - val_acc: 0.7727
Epoch 243/500
121/121 [==============================] - 0s 1ms/step - loss: 5.5008e-04 - acc: 1.0000 - val_loss: 0.5946 - val_acc: 0.7727
Epoch 244/500
121/121 [==============================] - 0s 1ms/step - loss: 5.4598e-04 - acc: 1.0000 - val_loss: 0.5943 - val_acc: 0.7727
Epoch 245/500
121/121 [==============================] - 0s 1ms/step - loss: 5.4095e-04 - acc: 1.0000 - val_loss: 0.5941 - val_acc: 0.7727
Epoch 246/500
121/121 [==============================] - 0s 1ms/step - loss: 5.3619e-04 - acc: 1.0000 - val_loss: 0.5948 - val_acc: 0.7727
Epoch 247/500
121/121 [==============================] - 0s 1ms/step - loss: 5.3137e-04 - acc: 1.0000 - val_loss: 0.5951 - val_acc: 0.7727
Epoch 248/500
121/121 [==============================] - 0s 1ms/step - loss: 5.2745e-04 - acc: 1.0000 - val_loss: 0.5954 - val_acc: 0.7727
Epoch 249/500
121/121 [==================

121/121 [==============================] - 0s 1ms/step - loss: 3.5267e-04 - acc: 1.0000 - val_loss: 0.5933 - val_acc: 0.7727
Epoch 302/500
121/121 [==============================] - 0s 1ms/step - loss: 3.5004e-04 - acc: 1.0000 - val_loss: 0.5937 - val_acc: 0.7727
Epoch 303/500
121/121 [==============================] - 0s 1ms/step - loss: 3.4761e-04 - acc: 1.0000 - val_loss: 0.5942 - val_acc: 0.7727
Epoch 304/500
121/121 [==============================] - 0s 1ms/step - loss: 3.4532e-04 - acc: 1.0000 - val_loss: 0.5944 - val_acc: 0.7727
Epoch 305/500
121/121 [==============================] - 0s 1ms/step - loss: 3.4295e-04 - acc: 1.0000 - val_loss: 0.5946 - val_acc: 0.7727
Epoch 306/500
121/121 [==============================] - 0s 1ms/step - loss: 3.4044e-04 - acc: 1.0000 - val_loss: 0.5949 - val_acc: 0.7727
Epoch 307/500
121/121 [==============================] - 0s 1ms/step - loss: 3.3843e-04 - acc: 1.0000 - val_loss: 0.5948 - val_acc: 0.7727
Epoch 308/500
121/121 [==================

121/121 [==============================] - 0s 1ms/step - loss: 2.4198e-04 - acc: 1.0000 - val_loss: 0.5949 - val_acc: 0.7727
Epoch 361/500
121/121 [==============================] - 0s 1ms/step - loss: 2.4061e-04 - acc: 1.0000 - val_loss: 0.5945 - val_acc: 0.7727
Epoch 362/500
121/121 [==============================] - 0s 1ms/step - loss: 2.3937e-04 - acc: 1.0000 - val_loss: 0.5956 - val_acc: 0.7727
Epoch 363/500
121/121 [==============================] - 0s 1ms/step - loss: 2.3770e-04 - acc: 1.0000 - val_loss: 0.5953 - val_acc: 0.7727
Epoch 364/500
121/121 [==============================] - 0s 1ms/step - loss: 2.3635e-04 - acc: 1.0000 - val_loss: 0.5950 - val_acc: 0.7727
Epoch 365/500
121/121 [==============================] - 0s 1ms/step - loss: 2.3515e-04 - acc: 1.0000 - val_loss: 0.5951 - val_acc: 0.7727
Epoch 366/500
121/121 [==============================] - 0s 1ms/step - loss: 2.3377e-04 - acc: 1.0000 - val_loss: 0.5941 - val_acc: 0.7727
Epoch 367/500
121/121 [==================

121/121 [==============================] - 0s 1ms/step - loss: 1.7491e-04 - acc: 1.0000 - val_loss: 0.5965 - val_acc: 0.7727
Epoch 420/500
121/121 [==============================] - 0s 1ms/step - loss: 1.7402e-04 - acc: 1.0000 - val_loss: 0.5963 - val_acc: 0.7727
Epoch 421/500
121/121 [==============================] - 0s 1ms/step - loss: 1.7304e-04 - acc: 1.0000 - val_loss: 0.5965 - val_acc: 0.7727
Epoch 422/500
121/121 [==============================] - 0s 1ms/step - loss: 1.7218e-04 - acc: 1.0000 - val_loss: 0.5969 - val_acc: 0.7727
Epoch 423/500
121/121 [==============================] - 0s 1ms/step - loss: 1.7124e-04 - acc: 1.0000 - val_loss: 0.5968 - val_acc: 0.7727
Epoch 424/500
121/121 [==============================] - 0s 1ms/step - loss: 1.7040e-04 - acc: 1.0000 - val_loss: 0.5972 - val_acc: 0.7727
Epoch 425/500
121/121 [==============================] - 0s 1ms/step - loss: 1.6955e-04 - acc: 1.0000 - val_loss: 0.5973 - val_acc: 0.7727
Epoch 426/500
121/121 [==================

121/121 [==============================] - 0s 1ms/step - loss: 1.3115e-04 - acc: 1.0000 - val_loss: 0.5986 - val_acc: 0.7727
Epoch 479/500
121/121 [==============================] - 0s 1ms/step - loss: 1.3048e-04 - acc: 1.0000 - val_loss: 0.5987 - val_acc: 0.7727
Epoch 480/500
121/121 [==============================] - 0s 1ms/step - loss: 1.2994e-04 - acc: 1.0000 - val_loss: 0.5986 - val_acc: 0.7727
Epoch 481/500
121/121 [==============================] - 0s 1ms/step - loss: 1.2935e-04 - acc: 1.0000 - val_loss: 0.5984 - val_acc: 0.7727
Epoch 482/500
121/121 [==============================] - 0s 1ms/step - loss: 1.2871e-04 - acc: 1.0000 - val_loss: 0.5989 - val_acc: 0.7727
Epoch 483/500
121/121 [==============================] - 0s 1ms/step - loss: 1.2816e-04 - acc: 1.0000 - val_loss: 0.5987 - val_acc: 0.7727
Epoch 484/500
121/121 [==============================] - 0s 1ms/step - loss: 1.2749e-04 - acc: 1.0000 - val_loss: 0.5986 - val_acc: 0.7727
Epoch 485/500
121/121 [==================

In [34]:
model.save('model.h5')

In [35]:
[np.argmax(i) for i in model.predict(X_test)][:10]

[0, 1, 10, 3, 4, 5, 6, 7, 8, 9]

In [36]:
[np.argmax(i) for i in y_test][:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]